# Compare Models

Create object to automated model comparisons

In [1]:
import arithmetic_datasets as ad

from tensorflow.keras.layers import Input, Lambda, Dense, concatenate
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
import random
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import tensorflow as tf
import random
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Metric

Using TensorFlow backend.


### ModelComparator

```
mc = ModelComparator()
mc.add_model(name, model)
mc.add_training_set(name, data)

mc.train_models(epochs, batches, **kwargs)

mc.compare_min(metric)
mc.compare_max(metric)

mc.history_for(model_name, training_set_name)
```

In [2]:
class ModelComparator:
    def __init__(self):
        self.models = {}
        self.training_sets = {}
        self.histories = {}
    
    def add_model(self, name, model):
        self.models[name] = model
    
    def add_training_set(self, name, data):
        self.training_sets[name] = data
        
    def train_models(self, **kwargs):
        for model_name in self.models:
            for tset_name in self.training_sets:
                print(f"MODEL {model_name} TRAINING ON DATASET {tset_name}")
                
                model = self.models[model_name]()
                
                history = model.fit(
                    self.training_sets[tset_name][0], self.training_sets[tset_name][1],
                    **kwargs
                )
                
                self.histories[(model_name, tset_name)] = history
    
    def compare_min(self, metric):
        result = {}
        minimum = None
        min_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amin(history[metric])
            if minimum == None or minimum >= result[hkey]:
                minimum = result[hkey]
                min_name = hkey
        return result, {"name" : min_name, "value":minimum}
            
    
    def compare_max(self, metric):
        result = {}
        maximum = None
        max_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amax(history[metric])
            if maximum == None or maximum >= result[hkey]:
                maximum = result[hkey]
                max_name = hkey
        return result, {"name" : max_name, "value": maximum}
    
    def history_for(self, model_name, training_set_name):
        return self.histories[(model_name, training_set_name)]

### Create Models

In [3]:
def nondense_model():
    # Input layer of 3 neurons 
    inp = Input(shape=(1,3))
    
    #128 layer
    d2_out = Dense(128)(inp)

    #grab first, 2nd half of the 128 layer
    d2_out_p1 = Lambda(lambda x: x[:,:,0:64])(d2_out)
    d2_out_p2 = Lambda(lambda x: x[:,:,64:128])(d2_out)

    #64 layer(s)
    d3_out = Dense(64)(d2_out_p1)
    d4_out = Dense(64)(d2_out_p2)

    #grab output nodes from both 64 layers
    d5_out = concatenate([d3_out, d4_out])
    
    o = Dense(1)(d5_out)
    
    model = Model(inp, o)

    model.compile(
        loss="MeanSquaredError",
        metrics=['accuracy']
    )
    
    return model

def dense_model_5L():
    model_5layer = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1024, input_shape=(1,3)),
        tf.keras.layers.Dense(512),
        tf.keras.layers.Dense(256),
        tf.keras.layers.Dense(128),
        tf.keras.layers.Dense(64),
        tf.keras.layers.Dense(1)
    ])
    
    model_5layer.compile(
        loss="MeanSquaredError",
        metrics=['accuracy'] #Acc not working, in testing
    )

    return model_5layer

### Datasets

In [5]:
num_examples = 20000
rstart = -50000
rend = 50000

add_setX, add_setY = ad.gen_data_add(num_examples, rstart, rend, integers_only=True)
sub_setX, sub_setY = ad.gen_data_sub(num_examples, rstart, rend, integers_only=True)

add_setX = add_setX.reshape(num_examples, 1, 3)
sub_setX = sub_setX.reshape(num_examples, 1, 3)

# 50-50
step = int(0.5*num_examples)

addsub_set1X = np.concatenate((add_setX[:step], sub_setX[step:]))
addsub_set1Y = np.concatenate((add_setY[:step], sub_setY[step:]))

# 80-20
step = int(0.8*num_examples)
addsub_set2X = np.concatenate((add_setX[:step], sub_setX[step:]))
addsub_set2Y = np.concatenate((add_setY[:step], sub_setY[step:]))

# MULT and DIV
mult_setX, mult_setY = ad.gen_data_mult(num_examples, rstart, rend, integers_only=True)
div_setX, div_setY = ad.gen_data_div(num_examples, rstart, rend, integers_only=True)

mult_setX = mult_setX.reshape(num_examples, 1, 3)
div_setX = div_setX.reshape(num_examples, 1, 3)

# MD 50-50
step = int(0.5*num_examples)
multdiv_set1X = np.concatenate((mult_setX[:step], div_setX[step:]))
multdiv_set1Y = np.concatenate((mult_setY[:step], div_setY[step:]))

# MD 80-20
step = int(0.8*num_examples)
multdiv_set2X = np.concatenate((mult_setX[:step], div_setX[step:]))
multdiv_set2Y = np.concatenate((mult_setY[:step], div_setY[step:]))

### Build ModelComparator

In [6]:
mc = ModelComparator()

mc.add_model("nondense_model", nondense_model)
mc.add_model("dense_model", dense_model_5L)

mc.add_training_set("addition", (add_setX, add_setY))
mc.add_training_set("subtraction", (sub_setX, sub_setY))
mc.add_training_set("add-sub 50/50", (addsub_set1X, addsub_set1Y))
mc.add_training_set("add-sub 80/20", (addsub_set2X, addsub_set2Y))
mc.add_training_set("multiplication", (mult_setX, mult_setY))
mc.add_training_set("division", (div_setX, div_setY))
mc.add_training_set("mult-div 50/50", (multdiv_set1X, multdiv_set1Y))
mc.add_training_set("mult-div 80/20", (multdiv_set2X, multdiv_set2Y))



### Train Models

In [7]:
num_epochs = 100
batch_size = num_examples

mc.train_models(
    batch_size=batch_size,
    epochs=num_epochs
)

MODEL nondense_model TRAINING ON DATASET addition
Epoch 1/100
1/1 [==============================] - 0s 16ms/step - loss: 2334297856.0000 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 997us/step - loss: 1445523456.0000 - accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 988525952.0000 - accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 687999040.0000 - accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 476209952.0000 - accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 323946752.0000 - accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 8ms/step - loss: 214838592.0000 - accuracy: 0.0000e+00
Epoch 8/100
1/1 [==============================] - 0s 996us/step - loss: 138011184.0000 - accuracy: 0.0000e+00
Epoch 9/100
1/1 [==============================] - 0s 3ms/step - loss: 

1/1 [==============================] - 0s 2ms/step - loss: 1325821.0000 - accuracy: 0.0000e+00
Epoch 78/100
1/1 [==============================] - 0s 997us/step - loss: 1409962.2500 - accuracy: 0.0000e+00
Epoch 79/100
1/1 [==============================] - 0s 997us/step - loss: 1668881.2500 - accuracy: 0.0000e+00
Epoch 80/100
1/1 [==============================] - 0s 2ms/step - loss: 2204397.2500 - accuracy: 0.0000e+00
Epoch 81/100
1/1 [==============================] - 0s 2ms/step - loss: 2939788.5000 - accuracy: 0.0000e+00
Epoch 82/100
1/1 [==============================] - 0s 998us/step - loss: 3839625.7500 - accuracy: 0.0000e+00
Epoch 83/100
1/1 [==============================] - 0s 2ms/step - loss: 4449538.5000 - accuracy: 0.0000e+00
Epoch 84/100
1/1 [==============================] - 0s 996us/step - loss: 4604856.5000 - accuracy: 0.0000e+00
Epoch 85/100
1/1 [==============================] - 0s 2ms/step - loss: 4212586.5000 - accuracy: 0.0000e+00
Epoch 86/100
1/1 [===============

1/1 [==============================] - 0s 2ms/step - loss: 4618095.0000 - accuracy: 0.0000e+00
Epoch 53/100
1/1 [==============================] - 0s 996us/step - loss: 2083956.5000 - accuracy: 5.0000e-05
Epoch 54/100
1/1 [==============================] - 0s 0s/step - loss: 1043728.5000 - accuracy: 0.0000e+00
Epoch 55/100
1/1 [==============================] - 0s 997us/step - loss: 631026.2500 - accuracy: 5.0000e-05
Epoch 56/100
1/1 [==============================] - 0s 3ms/step - loss: 469434.0625 - accuracy: 0.0000e+00
Epoch 57/100
1/1 [==============================] - 0s 997us/step - loss: 457322.2812 - accuracy: 5.0000e-05
Epoch 58/100
1/1 [==============================] - 0s 997us/step - loss: 631884.6875 - accuracy: 0.0000e+00
Epoch 59/100
1/1 [==============================] - 0s 997us/step - loss: 1387723.5000 - accuracy: 5.0000e-05
Epoch 60/100
1/1 [==============================] - 0s 2ms/step - loss: 4337264.5000 - accuracy: 0.0000e+00
Epoch 61/100
1/1 [==================

1/1 [==============================] - 0s 997us/step - loss: 826225152.0000 - accuracy: 0.0000e+00
Epoch 27/100
1/1 [==============================] - 0s 2ms/step - loss: 826224384.0000 - accuracy: 0.0000e+00
Epoch 28/100
1/1 [==============================] - 0s 997us/step - loss: 826223488.0000 - accuracy: 0.0000e+00
Epoch 29/100
1/1 [==============================] - 0s 2ms/step - loss: 826222656.0000 - accuracy: 0.0000e+00
Epoch 30/100
1/1 [==============================] - 0s 997us/step - loss: 826221888.0000 - accuracy: 0.0000e+00
Epoch 31/100
1/1 [==============================] - 0s 998us/step - loss: 826221120.0000 - accuracy: 0.0000e+00
Epoch 32/100
1/1 [==============================] - 0s 998us/step - loss: 826220224.0000 - accuracy: 0.0000e+00
Epoch 33/100
1/1 [==============================] - 0s 2ms/step - loss: 826219456.0000 - accuracy: 0.0000e+00
Epoch 34/100
1/1 [==============================] - 0s 2ms/step - loss: 826218624.0000 - accuracy: 0.0000e+00
Epoch 35/100


1/1 [==============================] - 0s 3ms/step - loss: 827996352.0000 - accuracy: 0.0000e+00
MODEL nondense_model TRAINING ON DATASET add-sub 80/20
Epoch 1/100
1/1 [==============================] - 0s 0s/step - loss: 1214572544.0000 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 762716608.0000 - accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 0s/step - loss: 621051264.0000 - accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 566209728.0000 - accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 0s/step - loss: 545523840.0000 - accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 997us/step - loss: 538387072.0000 - accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 2ms/step - loss: 536196064.0000 - accuracy: 0.0000e+00
Epoch 8/100
1/1 [==============================] - 0s 998us/step - loss: 535604192.00

...

Epoch 1/100
1/1 [==============================] - 0s 3ms/step - loss: 9444930.0000 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 3ms/step - loss: 34905837568.0000 - accuracy: 5.0000e-05
Epoch 3/100
1/1 [==============================] - 0s 3ms/step - loss: 4629785088.0000 - accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 4ms/step - loss: 138669936.0000 - accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 4ms/step - loss: 6851953.5000 - accuracy: 5.0000e-05
Epoch 6/100
1/1 [==============================] - 0s 3ms/step - loss: 527290.8750 - accuracy: 5.0000e-05
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 64897.7148 - accuracy: 5.0000e-05
Epoch 8/100
1/1 [==============================] - 0s 4ms/step - loss: 36911.5703 - accuracy: 5.0000e-05
Epoch 9/100
1/1 [==============================] - 0s 16ms/step - loss: 35693.7148 - accuracy: 5.0000e-05
Epoch 10/100
1/1 [================

1/1 [==============================] - 0s 16ms/step - loss: 37620.4961 - accuracy: 5.0000e-05
Epoch 78/100
1/1 [==============================] - 0s 16ms/step - loss: 37317.8672 - accuracy: 0.0000e+00
Epoch 79/100
1/1 [==============================] - 0s 0s/step - loss: 37415.7812 - accuracy: 5.0000e-05
Epoch 80/100
1/1 [==============================] - 0s 0s/step - loss: 37981.2227 - accuracy: 0.0000e+00
Epoch 81/100
1/1 [==============================] - 0s 0s/step - loss: 39388.0625 - accuracy: 5.0000e-05
Epoch 82/100
1/1 [==============================] - 0s 4ms/step - loss: 42865.8711 - accuracy: 0.0000e+00
Epoch 83/100
1/1 [==============================] - 0s 3ms/step - loss: 52084.6328 - accuracy: 5.0000e-05
Epoch 84/100
1/1 [==============================] - 0s 16ms/step - loss: 79937.0469 - accuracy: 0.0000e+00
Epoch 85/100
1/1 [==============================] - 0s 0s/step - loss: 173402.0156 - accuracy: 5.0000e-05
Epoch 86/100
1/1 [==============================] - 0s 0s/s

1/1 [==============================] - 0s 16ms/step - loss: 344750633938583552.0000 - accuracy: 5.0000e-05
Epoch 49/100
1/1 [==============================] - 0s 0s/step - loss: 344750118542508032.0000 - accuracy: 5.0000e-05
Epoch 50/100
1/1 [==============================] - 0s 16ms/step - loss: 344750599578845184.0000 - accuracy: 5.0000e-05
Epoch 51/100
1/1 [==============================] - 0s 0s/step - loss: 344750084182769664.0000 - accuracy: 5.0000e-05
Epoch 52/100
1/1 [==============================] - 0s 12ms/step - loss: 344750565219106816.0000 - accuracy: 5.0000e-05
Epoch 53/100
1/1 [==============================] - 0s 4ms/step - loss: 344750084182769664.0000 - accuracy: 5.0000e-05
Epoch 54/100
1/1 [==============================] - 0s 4ms/step - loss: 344750530859368448.0000 - accuracy: 5.0000e-05
Epoch 55/100
1/1 [==============================] - 0s 4ms/step - loss: 344749981103554560.0000 - accuracy: 5.0000e-05
Epoch 56/100
1/1 [==============================] - 0s 3ms/s

1/1 [==============================] - 0s 9ms/step - loss: 555382324124975104.0000 - accuracy: 0.0000e+00
Epoch 17/100
1/1 [==============================] - 0s 0s/step - loss: 555381808728899584.0000 - accuracy: 0.0000e+00
Epoch 18/100
1/1 [==============================] - 0s 0s/step - loss: 555382392844451840.0000 - accuracy: 0.0000e+00
Epoch 19/100
1/1 [==============================] - 0s 0s/step - loss: 555381774369161216.0000 - accuracy: 0.0000e+00
Epoch 20/100
1/1 [==============================] - 0s 0s/step - loss: 555382152326283264.0000 - accuracy: 0.0000e+00
Epoch 21/100
1/1 [==============================] - 0s 16ms/step - loss: 555381671289946112.0000 - accuracy: 0.0000e+00
Epoch 22/100
1/1 [==============================] - 0s 0s/step - loss: 555382117966544896.0000 - accuracy: 0.0000e+00
Epoch 23/100
1/1 [==============================] - 0s 4ms/step - loss: 555381671289946112.0000 - accuracy: 0.0000e+00
Epoch 24/100
1/1 [==============================] - 0s 4ms/step -

1/1 [==============================] - 0s 0s/step - loss: 555380399979626496.0000 - accuracy: 0.0000e+00
Epoch 86/100
1/1 [==============================] - 0s 3ms/step - loss: 555380881015963648.0000 - accuracy: 0.0000e+00
Epoch 87/100
1/1 [==============================] - 0s 16ms/step - loss: 555380399979626496.0000 - accuracy: 0.0000e+00
Epoch 88/100
1/1 [==============================] - 0s 0s/step - loss: 555380881015963648.0000 - accuracy: 0.0000e+00
Epoch 89/100
1/1 [==============================] - 0s 0s/step - loss: 555380262540673024.0000 - accuracy: 0.0000e+00
Epoch 90/100
1/1 [==============================] - 0s 0s/step - loss: 555380743577010176.0000 - accuracy: 0.0000e+00
Epoch 91/100
1/1 [==============================] - 0s 16ms/step - loss: 555380193821196288.0000 - accuracy: 0.0000e+00
Epoch 92/100
1/1 [==============================] - 0s 0s/step - loss: 555380709217271808.0000 - accuracy: 0.0000e+00
Epoch 93/100
1/1 [==============================] - 0s 16ms/step

### compare min loss

In [8]:
models_min_loss, min_loss = mc.compare_min("loss")

In [9]:
models_min_loss

{('nondense_model', 'addition'): 13.663665771484375,
 ('nondense_model', 'subtraction'): 172.84950256347656,
 ('nondense_model', 'add-sub 50/50'): 826217344.0,
 ('nondense_model', 'add-sub 80/20'): 535423232.0,
 ('nondense_model', 'multiplication'): 6.89402105672237e+17,
 ('nondense_model', 'division'): 383841.53125,
 ('nondense_model', 'mult-div 50/50'): 3.4477928996038246e+17,
 ('nondense_model', 'mult-div 80/20'): 5.5541245761552384e+17,
 ('dense_model', 'addition'): 988200.875,
 ('dense_model', 'subtraction'): 74544.4765625,
 ('dense_model', 'add-sub 50/50'): 826182144.0,
 ('dense_model', 'add-sub 80/20'): 535433952.0,
 ('dense_model', 'multiplication'): 6.893291943074202e+17,
 ('dense_model', 'division'): 35493.953125,
 ('dense_model', 'mult-div 50/50'): 3.447491564698337e+17,
 ('dense_model', 'mult-div 80/20'): 5.553800907419812e+17}